最大シーケンス長に分割されたJOSONLに整形

In [ ]:
import json
import path as path

output_file = path.wiki_batched
with open(output_file, 'w', encoding='utf-8') as json_file:
    stack = ""
    with open(path.wiki, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            if len(data["text"]) <= 10: continue
            elif len(data["text"]) <= path.max_seq_len // 2:
                stack += data["text"] + "<pad>"
                if len(stack) > path.max_seq_len // 2:
                    json.dump({"text": stack}, json_file, ensure_ascii=False)
                    json_file.write('\n')
                    stack = ""
            elif len(data["text"]) > path.max_seq_len:
                split_index = [0]
                split_num = len(data["text"]) // path.max_seq_len + 1
                split_size = len(data["text"]) // split_num
                for i in range(split_num-1):
                    split_index.append(data["text"].rfind("。", split_index[i], split_index[i] + split_size)+1)
                    json.dump({"text": data["text"][split_index[i]:split_index[i+1]]}, json_file, ensure_ascii=False)
                    json_file.write('\n')
                json.dump({"text": data["text"][split_index[-1]:]}, json_file, ensure_ascii=False)
                json_file.write('\n')
            else:
                json.dump({"text": data["text"]}, json_file, ensure_ascii=False)
                json_file.write('\n')
            #break

データセットの作成しPT形式で保存

In [ ]:
import json
import path as path
from charTokenizer import CharTokenizer
from dataset import JPNDataset
import torch
from torch.utils.data import DataLoader

tokenizer = CharTokenizer()
with open(path.charTokenizer, 'r', encoding='utf-8') as f:
    tokenizer.vocab = json.load(f)

dataset = JPNDataset(path.wiki_batched, tokenizer, max_seq_len=path.max_seq_len)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
batch = next(iter(dataloader))
print(batch)

torch.save(dataset, f"../data/dataset_{path.max_seq_len}.pt")